<a href="https://colab.research.google.com/github/robertpetre/DSM/blob/main/DSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("cjinny/mura-v11")

print("Path to dataset files:", path)

100%|██████████| 3.14G/3.14G [00:36<00:00, 92.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/cjinny/mura-v11/versions/1


In [10]:
import os
import pandas as pd

In [29]:
root_path = "/root/.cache/kagglehub/datasets/cjinny/mura-v11/versions/1/MURA-v1.1/"

csv_path_images = os.path.join(root_path, "train_image_paths.csv")
csv_path_labels = os.path.join(root_path, "train_labeled_studies.csv")

df_images = pd.read_csv(csv_path_images)

filtered_df = df_images[df_images.iloc[:, 0].apply(lambda x: x.split("/")[2] == "XR_FOREARM")]

train_image_paths = filtered_df.iloc[:, 0].values


MURA-v1.1/train/XR_FOREARM/patient09083/study1_positive/image1.png


XR_SHOULDER
